In [3]:
!pip install scapy

     ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
     - -------------------------------------- 0.0/1.3 MB 219.4 kB/s eta 0:00:06
     - -------------------------------------- 0.1/1.3 MB 297.7 kB/s eta 0:00:05
     -- ------------------------------------- 0.1/1.3 MB 353.1 kB/s eta 0:00:04
     ----------------- ---------------------- 0.6/1.3 MB 2.0 MB/s eta 0:00:01
     ---------------------------------------  1.3/1.3 MB 4.2 MB/s eta 0:00:01
     ---------------------------------------- 1.3/1.3 MB 3.9 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished wi

ERROR: Could not install packages due to an OSError: [WinError 5] 액세스가 거부되었습니다: 'C:\\Python312\\share'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
#Importing the necessary modules
import logging
from datetime import datetime
import subprocess
import sys
 
#This will suppress all messages that have a lower level of seriousness than error messages, while running or loading Scapy
logging.getLogger("scapy.runtime").setLevel(logging.ERROR)
logging.getLogger("scapy.interactive").setLevel(logging.ERROR)
logging.getLogger("scapy.loading").setLevel(logging.ERROR)
 
 
try:
    from scapy.all import *
 
except ImportError:
    print("Scapy package for Python is not installed on your system.")
    sys.exit()

Scapy package for Python is not installed on your system.


AttributeError: 'tuple' object has no attribute 'tb_frame'

In [ ]:
#Asking the user for some parameters: interface on which to sniff, the number of packets to sniff, the time interval to sniff, the protocol
 
#Asking the user for input - the interface on which to run the sniffer
net_iface = input("* Enter the interface on which to run the sniffer (e.g. 'enp0s8'): ")
 
#Setting network interface in promiscuous mode
'''
Wikipedia: In computer networking, promiscuous mode or "promisc mode"[1] is a mode for a wired network interface controller (NIC) or wireless network interface controller (WNIC) that causes the controller to pass all traffic it receives to the central processing unit (CPU) rather than passing only the frames that the controller is intended to receive.
This mode is normally used for packet sniffing that takes place on a router or on a computer connected to a hub.
'''
try:
    subprocess.call(["ifconfig", net_iface, "promisc"], stdout = None, stderr = None, shell = False)
 
except:
    print("\nFailed to configure interface as promiscuous.\n")
 
else:
    #Executed if the try clause does not raise an exception
    print("\nInterface %s was set to PROMISC mode.\n" % net_iface)
 
 
#Asking the user for the number of packets to sniff (the "count" parameter)
pkt_to_sniff = input("* Enter the number of packets to capture (0 is infinity): ")
 
#Considering the case when the user enters 0 (infinity)
if int(pkt_to_sniff) != 0:
    print("\nThe program will capture %d packets.\n" % int(pkt_to_sniff))
    
elif int(pkt_to_sniff) == 0:
    print("\nThe program will capture packets until the timeout expires.\n")
 
 
#Asking the user for the time interval to sniff (the "timeout" parameter)
time_to_sniff = input("* Enter the number of seconds to run the capture: ")
 
#Handling the value entered by the user
if int(time_to_sniff) != 0:
    print("\nThe program will capture packets for %d seconds.\n" % int(time_to_sniff))
    
    
#Asking the user for any protocol filter he might want to apply to the sniffing process
#For this example I chose three protocols: ARP, BOOTP, ICMP
#You can customize this to add your own desired protocols
proto_sniff = input("* Enter the protocol to filter by (arp|bootp|icmp|0 is all): ")
 
#Considering the case when the user enters 0 (meaning all protocols)
if (proto_sniff == "arp") or (proto_sniff == "bootp") or (proto_sniff == "icmp"):
    print("\nThe program will capture only %s packets.\n" % proto_sniff.upper())
    
elif (proto_sniff) == "0":
    print("\nThe program will capture all protocols.\n")
 
 
#Asking the user to enter the name and path of the log file to be created
file_name = input("* Please give a name to the log file: ")
 
#Creating the text file (if it doesn't exist) for packet logging and/or opening it for appending
sniffer_log = open(file_name, "a")

In [ ]:
#This is the function that will be called for each captured packet
#The function will extract parameters from the packet and then log each packet to the log file
def packet_log(packet):
    
    #Getting the current timestamp
    now = datetime.now()
    
    #Writing the packet information to the log file, also considering the protocol or 0 for all protocols
    if proto_sniff == "0":
        #Writing the data to the log file
        print("Time: " + str(now) + " Protocol: ALL" + " SMAC: " + packet[0].src + " DMAC: " + packet[0].dst, file = sniffer_log)
        
    elif (proto_sniff == "arp") or (proto_sniff == "bootp") or (proto_sniff == "icmp"):
        #Writing the data to the log file
        print("Time: " + str(now) + " Protocol: " + proto_sniff.upper() + " SMAC: " + packet[0].src + " DMAC: " + packet[0].dst, file = sniffer_log)

In [ ]:
#Importing the necessary modules
import difflib
import datetime
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from netmiko import ConnectHandler
 
 
#Defining the device to monitor
ip = '10.10.10.2'
 
#Defining the device type for running netmiko
device_type = 'arista_eos'
 
#Defining the username and password for running netmiko
username = 'admin'
password = 'python'
 
#Defining the command to send to each device
command = 'show running'